**HAND GESTURE-BASED INTERACTION**

---

Group members:
*   Ada Yılmaz
*   Ceren Şahin
*   List item
*   List item

# Installing necessary libraries and models

In [1]:
#install mediapipe
!pip install -q mediapipe

In [2]:
#download a model that can recognize 7 hand gestures: 👍, 👎, ✌️, ☝️, ✊, 👋, 🤟
!wget -q https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/1/gesture_recognizer.task

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
#download test images from pixabay
import urllib

IMAGE_FILENAMES = ['thumbs_down.jpg', 'victory.jpg', 'thumbs_up.jpg', 'pointing_up.jpg']

for name in IMAGE_FILENAMES:
  url = f'https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/{name}'
  urllib.request.urlretrieve(url, name)

In [4]:
#or we can use our own images as shown below

# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)
# IMAGE_FILENAMES = list(uploaded.keys())

# print('Uploaded files:', IMAGE_FILENAMES)

#Functions for visualization

In [1]:
#some functions to visualize the gesture recognition results.
import math
from matplotlib import pyplot as plt
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

plt.rcParams.update({
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'xtick.labelbottom': False,
    'xtick.bottom': False,
    'ytick.labelleft': False,
    'ytick.left': False,
    'xtick.labeltop': False,
    'xtick.top': False,
    'ytick.labelright': False,
    'ytick.right': False
})

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


def display_one_image(image, title, subplot, titlesize=16):
    """Displays one image along with the predicted category name and score."""
    plt.subplot(*subplot)
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize), color='black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)


def display_batch_of_images_with_gestures_and_hand_landmarks(images, results):
    """Displays a batch of images with the gesture category and its score along with the hand landmarks."""
    # Images and labels.
    images = [image.numpy_view() for image in images]
    gestures = [top_gesture for (top_gesture, _) in results]
    multi_hand_landmarks_list = [multi_hand_landmarks for (_, multi_hand_landmarks) in results]

    # Auto-squaring: this will drop data that does not fit into square or square-ish rectangle.
    rows = int(math.sqrt(len(images)))
    cols = len(images) // rows

    # Size and spacing.
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))

    # Display gestures and hand landmarks.
    for i, (image, gestures) in enumerate(zip(images[:rows*cols], gestures[:rows*cols])):
        title = f"{gestures.category_name} ({gestures.score:.2f})"
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols) * 40 + 3
        annotated_image = image.copy()

        for hand_landmarks in multi_hand_landmarks_list[i]:
          hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
          hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
          ])

          mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        subplot = display_one_image(annotated_image, title, subplot, titlesize=dynamic_titlesize)

    # Layout.
    plt.tight_layout()
    plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

#Preview images

In [2]:
import cv2

from google.colab.patches import cv2_imshow
import math

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2_imshow(img)


# Preview the images.
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}
for name, image in images.items():
  print(name)
  resize_and_show(image)

ModuleNotFoundError: No module named 'google.colab'

#Running

this first one is a how to from google, we can create our own by following the steps given.


In [3]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an GestureRecognizer object.
base_options = python.BaseOptions(model_asset_path='gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

images = []
results = []
for image_file_name in IMAGE_FILENAMES:
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_file_name)

  # STEP 4: Recognize gestures in the input image.
  recognition_result = recognizer.recognize(image)

  # STEP 5: Process the result. In this case, visualize it.
  images.append(image)
  top_gesture = recognition_result.gestures[0][0]
  hand_landmarks = recognition_result.hand_landmarks
  results.append((top_gesture, hand_landmarks))

display_batch_of_images_with_gestures_and_hand_landmarks(images, results)

RuntimeError: Unable to open file at c:\Users\alync\Documents\GitHub\gesture-based-interaction\gesture_recognizer.task

i tried as an example here.

In [4]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

def detect_peace_sign(hand_landmarks): #example peace gesture
    # Get coordinates of relevant landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    # Define peace sign gesture: index and middle fingers up, ring and pinky fingers down
    if (index_tip.y < middle_tip.y < ring_tip.y and pinky_tip.y > ring_tip.y):
        return True
    return False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect custom gesture (peace sign)
            if detect_peace_sign(hand_landmarks):
                cv2.putText(frame, 'Peace Sign Detected!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                # Trigger interaction (e.g., print a message)
                print("Peace sign gesture detected!")

    # Display the frame
    cv2.imshow('MediaPipe Hands', frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign gesture detected!
Peace sign ges

In [10]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Gesture detection functions
def detect_peace_sign(hand_landmarks):
    """Detect a peace sign gesture (index and middle fingers up, others down)."""
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    
    # Logic for peace sign
    if (index_tip.y < middle_tip.y < ring_tip.y and pinky_tip.y > ring_tip.y):
        return True
    return False

def detect_thumbs_up(hand_landmarks):
    """Detect a thumbs-up gesture."""
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

    # Logic for thumbs-up
    if thumb_tip.y < index_tip.y and thumb_tip.x < thumb_mcp.x:
        return True
    return False

def detect_thumbs_down(hand_landmarks):
    """Detect a thumbs-down gesture."""
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

    # Logic for thumbs-down
    if thumb_tip.y > index_tip.y and thumb_tip.x < thumb_mcp.x:
        return True
    return False

# Initialize webcam
cap = cv2.VideoCapture(0)

print("Press 'Esc' to exit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect gestures
            if detect_peace_sign(hand_landmarks):
                cv2.putText(frame, 'Peace Sign Detected!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                print("Peace sign gesture detected!")
            elif detect_thumbs_up(hand_landmarks):
                cv2.putText(frame, 'Thumbs Up Detected!', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                print("Thumbs-up gesture detected!")
            elif detect_thumbs_down(hand_landmarks):
                cv2.putText(frame, 'Thumbs Down Detected!', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                print("Thumbs-down gesture detected!")

    # Display the frame
    cv2.imshow('Gesture Recognition', frame)

    # Break the loop when 'Esc' key is pressed
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Press 'Esc' to exit.
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture detected!
Thumbs-down gesture